In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
from polimorfo.datasets import CocoDataset
from polimorfo.utils import maskutils
from tqdm import tqdm
import pandas as pd
import numpy as np

In [4]:
ds_path = '../../car-models/datasets/scratches/val_scratches.json'

In [23]:
gt_ds = CocoDataset(ds_path)
gt_ds.reindex()
pred_ds = CocoDataset(ds_path)
pred_ds.reindex()

load annotations: 100%|██████████| 657/657 [00:00<00:00, 571239.16it/s]
reindex images: 998it [00:00, 768904.37it/s]
reindex annotations: 657it [00:00, 593508.02it/s]
load annotations: 100%|██████████| 657/657 [00:00<00:00, 774931.87it/s]
reindex images: 998it [00:00, 478171.74it/s]
reindex annotations: 657it [00:00, 413886.71it/s]


In [24]:
gt_ds.remove_annotations([1])
gt_ds.reindex()

reindex images: 998it [00:00, 630884.01it/s]
reindex annotations: 656it [00:00, 682575.89it/s]
reindex images: 998it [00:00, 300410.18it/s]
reindex annotations: 656it [00:00, 532506.95it/s]


In [52]:
header = [
        'img_path', 'gt_ann_id', 'pred_ann_id', 'true_class_id',
        'pred_class_id', 'intersection', 'union', 'IOU', 'score'
    ]

In [53]:
def best_match(pred_anns, gt_ann_id, gt_mask, img_path, gt_class_id):
    best_pred_ann_id = -1
    best_iou = 0
    best_values = [img_path, gt_ann_id, -1, gt_class_id, 0, 0, 0, 0, 0]
    for pred_ann in pred_anns:
        pred_mask = maskutils.polygons_to_mask(pred_ann['segmentation'], 
                                         gt_img_meta['height'],
                                        gt_img_meta['width'])
        pred_ann_id = pred_ann['id']
        pred_class_id = pred_ann['category_id']
        pred_score = pred_ann['score'] if 'score' in pred_ann else 1

        intersection = (pred_mask * gt_mask).sum()
        union = np.count_nonzero(pred_mask + gt_mask)
        iou = intersection / union
        
        if iou > best_iou:
            best_values = [img_path, gt_ann_id, pred_ann_id, gt_class_id,
                           pred_class_id, intersection, union, iou, pred_score]
            best_pred_ann_id = pred_ann_id
            best_iou = iou
    return best_pred_ann_id, best_values

In [54]:
results = []

for img_idx, gt_img_meta in tqdm(gt_ds.imgs.items()):
    gt_anns = gt_ds.get_annotations(img_idx)
    pred_img_meta = pred_ds.imgs[img_idx]
    
    if gt_img_meta['file_name'] != pred_img_meta['file_name']:
        raise Exception("images path compared are different")
        
    img_path = gt_img_meta['file_name']
        
    pred_anns = pred_ds.get_annotations(img_idx)
    
    # create a set with all the prediction that will be used to find FP
    pred_idx_dict = {ann['id']: ann for ann in pred_anns}
    
    for gt_ann in gt_anns:
        gt_mask = maskutils.polygons_to_mask(gt_ann['segmentation'], 
                                             gt_img_meta['height'],
                                            gt_img_meta['width'])
        gt_ann_id = gt_ann['id']
        gt_class_id = gt_ann['category_id']
        
        pred_ann_id, row = best_match(pred_anns, gt_ann_id, gt_mask, img_path, gt_class_id)
        results.append(row)
        if pred_ann_id in pred_idx_dict:
            del pred_idx_dict[pred_ann_id]
            pred_anns = pred_idx_dict.values()
                
    # false positive dict    
    for pred_ann_id, pred_ann in pred_idx_dict.items():
        results.append([img_path, -1, pred_ann_id, 0,
                               pred_ann['category_id'], 0, 0, 0, 0])
    

100%|██████████| 998/998 [00:01<00:00, 600.94it/s]


In [55]:
df = pd.DataFrame(results, columns=header)

In [56]:
df

,img_path,gt_ann_id,pred_ann_id,true_class_id,pred_class_id,intersection,union,IOU,score
0,batch6__2017010400696400__foto0004.jpg,0,0,1,1,968,968,1.0,1
1,batch6__2017010400696400__foto0004.jpg,-1,1,0,1,0,0,0.0,0
2,batch6__2017010400054500__foto0005.jpg,1,2,1,1,14315,14315,1.0,1
3,batch6__2017010400885200__foto0001.jpg,2,3,1,1,110,110,1.0,1
4,batch6__2017010400193000__foto0011.jpg,3,4,1,1,8267,8267,1.0,1
...,...,...,...,...,...,...,...,...,...
652,batch6__2017010400901900__foto0004.jpg,651,652,1,1,2252,2252,1.0,1
653,batch6__2017010400901900__foto0004.jpg,652,653,1,1,4109,4109,1.0,1
654,batch6__2017010400901900__foto0004.jpg,653,654,1,1,2657,2657,1.0,1
655,batch6__2017010400901900__foto0004.jpg,654,655,1,1,2488,2488,1.0,1


In [30]:
df['IOU'].mean()

0.9984779299847792

In [65]:
class_idxs = sorted(df['true_class_id'].unique())[1:]
for class_idx in class_idxs:
    df_class = df[(df['true_class_id'] == class_idx) | (df['pred_class_id'] == class_idx)]
    

In [66]:
df_class

,img_path,gt_ann_id,pred_ann_id,true_class_id,pred_class_id,intersection,union,IOU,score
0,batch6__2017010400696400__foto0004.jpg,0,0,1,1,968,968,1.0,1
1,batch6__2017010400696400__foto0004.jpg,-1,1,0,1,0,0,0.0,0
2,batch6__2017010400054500__foto0005.jpg,1,2,1,1,14315,14315,1.0,1
3,batch6__2017010400885200__foto0001.jpg,2,3,1,1,110,110,1.0,1
4,batch6__2017010400193000__foto0011.jpg,3,4,1,1,8267,8267,1.0,1
...,...,...,...,...,...,...,...,...,...
652,batch6__2017010400901900__foto0004.jpg,651,652,1,1,2252,2252,1.0,1
653,batch6__2017010400901900__foto0004.jpg,652,653,1,1,4109,4109,1.0,1
654,batch6__2017010400901900__foto0004.jpg,653,654,1,1,2657,2657,1.0,1
655,batch6__2017010400901900__foto0004.jpg,654,655,1,1,2488,2488,1.0,1


In [40]:
at_iou = 0.5

In [57]:
true_positives = df[(df['true_class_id'] == df['pred_class_id']) & (df['IOU'] > at_iou)]
# all the prediction that do not have a valid gt annotation
false_positives = df[df['gt_ann_id'] == -1]
# all the gt annotations that do not have a prediction
false_negatives = df[df['pred_ann_id'] == -1]

In [58]:
precision = len(true_positives) / (len(true_positives) + len(false_positives))
precision

0.9984779299847792

In [59]:
recall = len(true_positives) / (len(true_positives) + len(false_negatives))
recall

1.0

In [67]:
a = np.array([-1,0,1,2,3])

In [68]:
a[a >0]

array([1, 2, 3])

In [69]:
df['img_path'].loc[0]

'batch6__2017010400696400__foto0004.jpg'